https://www.winemag.com/?s=&drink_type=wine&page=1&search_type=reviews

In [1]:
from bs4 import NavigableString, Comment
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests

## Gather 

In [2]:
URL = 'https://www.wine.com/list/wine/7155?ratingmin=94#'

In [3]:
# this will be the list containing all the scraped information, 
# which will turn into a dataframe
web_data = []

def get_info(all_li):
    for li in all_li:
        product_name = li.find('span', class_= 'prodItemInfo_name').string
        price = li.find('span', class_='productPrice_price-regWhole').string
        varietal = li.find('span', class_='prodItemInfo_varietal').string
        origin = li.find('span', class_='prodItemInfo_originText').string
        rating = li.find('span', class_='averageRating_average').string
        rating_count = li.find('span', class_='averageRating_number').string
        web_data.append({'product_name' : product_name,
                    'price': price,
                    'varietal': varietal,
                    'origin': origin,
                    'rating': rating,
                    'rating_count': rating_count})

## Pagination Stuff 

In [4]:
url_df = []

def url_contructor(page_number, url):
    first = URL.split('7155', 1)[0]
    second = URL.split('7155',1)[1]
    url = first + '7155/' + str(page_number) + second
    url_df.append(url)

In [15]:
def pagination():
    page = 0
    URL = 'https://www.wine.com/list/wine/7155?ratingmin=94#'
    for x in range(1, 950):
        try:
            page += 1
            url_contructor(page, URL)
        except Exception:
            print(Exception)

## Link it all together 

In [16]:
# this is my main function that will execute the request and the url_contructor for pagination
def main_execute():
    pagination()
    for link in url_df:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        all_li = soup.find_all('li', {'class': 'prodItem'})
        get_info(all_li)

In [17]:
main_execute()

## Results

In [19]:
type(web_data)

list

In [20]:
df = pd.DataFrame(web_data)

In [21]:
df.shape

(23822, 6)

In [22]:
df

,product_name,price,varietal,origin,rating,rating_count
0,Dom Perignon Vintage with Gift Box 2010,199,Vintage Sparkling Wine,"Champagne, France",4.5,42
1,Veuve Clicquot Yellow Label Brut,59,Non-Vintage Sparkling Wine,"Champagne, France",4.4,1138
2,Duckhorn Napa Valley Cabernet Sauvignon 2017,78,Cabernet Sauvignon,"Napa Valley, California",4.3,62
3,Caymus Special Selection Cabernet Sauvignon 2016,180,Cabernet Sauvignon,"Napa Valley, California",4.6,108
4,Quintessa 2016,199,Cabernet Sauvignon,"Rutherford, Napa Valley, California",4.7,73
...,...,...,...,...,...,...
23817,Pax Alder Springs Vineyard The Knob 2005,59,Syrah/Shiraz,"Mendocino, California",0.0,0
23818,Williams Selyem Peay Vineyard Pinot Noir 2007,100,Pinot Noir,"Sonoma County, California",0.0,0
23819,Williams Selyem Allen Vineyard Chardonnay 2007,99,Chardonnay,"Russian River, Sonoma County, California",0.0,0
23820,Sine Qua Non Atlantis Fe203 3e Roussanne (375M...,299,Other Dessert,California,0.0,0


In [23]:
df.to_csv('wine_scraped_dataset.csv')